In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [ ]:
train_raw = pd.read_csv("drive/MyDrive/CSVALL/train.csv", names=['headline', 'body', 'label'])
train_raw.head()

,headline,body,label
0,Politico Reporter Drunkenly Unloads On Trump A...,Politico Playbook writer Anna Palmer pulled a ...,0
1,"CNN ’ s Jim Acosta Hits Back At Sean Hannity ,...",CNN White House correspondent Jim Acosta found...,1
2,Is This The Biggest Climate Change Scam Yet ?,Among the many reasons conservatives favor sma...,0
3,Trump Wins – Supreme Court Reinstates Presiden...,So much winning . <EOS> The Supreme Court is l...,1
4,Sequoia Forest ’ s ‘ National Monument ’ Statu...,The Portersville City Council has approved a d...,0


In [ ]:
def body_to_para(body, max_para = 50):
    para_mask = np.zeros(max_para)
    paras = body.split("<EOP>")
    if len(paras) >= max_para:
        return np.array([np.array(paras[:max_para]), para_mask])
    else:
        pad_by = max_para - len(paras)
        para_mask[-pad_by:] = np.ones(pad_by)
        for i in range(pad_by):
            paras.append("")
        ans =  np.vstack([np.array(paras), para_mask])
        return ans

In [ ]:
def bertify(apply_ans):
    embeddings = []
    para_masks = []
    for i in range(len(apply_ans)):
        embeddings.append(model.encode(apply_ans.iloc[i][0], device = 'cuda'))
        para_masks.append(apply_ans.iloc[i][1])

    return np.array(embeddings), np.array(para_masks)

In [ ]:
class ParaHeadlineAttention(nn.Module):
    def __init__(self, para_dim, headline_dim, hidden_dim):
        super().__init__()
        self.linear_para = nn.Linear(para_dim, hidden_dim, bias=False)
        self.linear_headline = nn.Linear(headline_dim, hidden_dim, bias=False)
        self.v = nn.Parameter(torch.rand(hidden_dim))  # [H_hidden]

    def forward(self, paras, para_mask, headline):
        # paras: [N, P, H_para]
        # para_mask: [N, P]
        # headline: [N, H_headline]
        z = self.linear_para(paras) + self.linear_headline(headline).unsqueeze(1)  # [N, P, H_hidden]
        s = torch.tanh(z).matmul(self.v)  # [N, P]
        s[para_mask] = float('-inf')
        a = torch.softmax(s, dim=1)  # [N, P]
        return torch.matmul(a.unsqueeze(1), paras).squeeze()  # paras: [N, H_para]

class FakeNewsClassifier(nn.Module):
    def __init__(self, hidden_dims_para, hidden_dims_headline):
        super().__init__()
        self.attention = ParaHeadlineAttention(hidden_dims_para, hidden_dims_headline, hidden_dims_para)
        self.bilinear = nn.Bilinear(hidden_dims_headline, hidden_dims_para, 1)

    def forward(self, bodys, para_masks, headlines):
        # headline: [N, L_headline], 
        # headline_lengths: [N],
        # bodys: [N, P, L_para], 
        # para_lengths: [N, P]
        
        h_body = self.attention(bodys, para_masks, headlines)
        return self.bilinear(headlines, h_body).squeeze()

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
    print("Light Speed...")
else:
    device = 'cpu'
    print("Slow Speed, Please connect to CUDA")

Light Speed...


In [ ]:
classifier = FakeNewsClassifier(384, 384)
if device=="cuda":
    classifier.cuda()
    model.cuda()
else:
    print("Please run on GPU.")
def init_weights(m):
    if type(m) == torch.nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)

classifier.apply(init_weights)

batch_size = 20
lr = 0.1
epochs = 1
classifier.train()

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=lr)

for epoch in range(epochs):
    for i in range(0, train_raw.shape[0], batch_size):
      batch = train_raw.iloc[i:min(train_raw.shape[0], i+batch_size)]
      batch.reset_index(inplace = True)
      headlines = (model.encode(batch['headline'], device = 'cuda'))
      headlines = torch.Tensor(headlines).cuda()
      body_mask = batch['body'].apply(body_to_para)
      bert_ans, para_masks = bertify(body_mask)
      bert_ans = torch.Tensor(bert_ans.astype(np.float)).cuda()
      para_masks = torch.Tensor(para_masks.astype(np.float).astype(np.int).astype(np.bool)).cuda()
      para_masks = para_masks.bool()
      y_pred = classifier(bert_ans, para_masks, headlines)
      print(torch.max(y_pred), ":::", torch.min(y_pred))
      loss = criterion(y_pred, torch.Tensor(batch['label']).cuda())
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
    print("Epoch : {}, Loss : {}".format(epoch, loss))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  if __name__ == '__main__':


tensor(0.0223, device='cuda:0', grad_fn=<MaxBackward1>) ::: tensor(-0.0484, device='cuda:0', grad_fn=<MinBackward1>)
tensor(0.9229, device='cuda:0', grad_fn=<MaxBackward1>) ::: tensor(-1.8041, device='cuda:0', grad_fn=<MinBackward1>)
tensor(4.8267, device='cuda:0', grad_fn=<MaxBackward1>) ::: tensor(-0.8751, device='cuda:0', grad_fn=<MinBackward1>)
tensor(5.9888, device='cuda:0', grad_fn=<MaxBackward1>) ::: tensor(-2.0017, device='cuda:0', grad_fn=<MinBackward1>)
tensor(2.8322, device='cuda:0', grad_fn=<MaxBackward1>) ::: tensor(-3.0913, device='cuda:0', grad_fn=<MinBackward1>)
tensor(3.4699, device='cuda:0', grad_fn=<MaxBackward1>) ::: tensor(-3.6764, device='cuda:0', grad_fn=<MinBackward1>)
tensor(2.5293, device='cuda:0', grad_fn=<MaxBackward1>) ::: tensor(-4.5404, device='cuda:0', grad_fn=<MinBackward1>)
tensor(2.5069, device='cuda:0', grad_fn=<MaxBackward1>) ::: tensor(-5.8668, device='cuda:0', grad_fn=<MinBackward1>)
tensor(5.8110, device='cuda:0', grad_fn=<MaxBackward1>) ::: tens

In [ ]:
# bert_ans, para_masks, headlines = torch.Tensor(bert_ans), torch.Tensor(para_masks.astype(np.float).astype(np.int).astype(np.bool)), torch.Tensor(headlines)

In [ ]:
# para_masks = para_masks.bool()

In [ ]:
# print(bert_ans.shape, para_masks.shape, headlines.shape)

torch.Size([10, 50, 384]) torch.Size([10, 50]) torch.Size([10, 384])


tensor([0.0238, 0.0302, 0.0131, 0.0152, 0.0100, 0.0029, 0.0117, 0.0411, 0.0195,
        0.0490], grad_fn=<SqueezeBackward0>)

In [ ]:
test_raw = pd.read_csv("drive/MyDrive/CSVALL/test.csv", names=['headline', 'body', 'label'])
test_raw.head()

,headline,body,label
6578,Pence says Trump will keep his economic promises,Vice President Mike Pence is adamant that Pres...,0
6579,Police name two of the London Bridge terrorists,Two of the three London attackers were identif...,1
6580,Jimmy Carter Shakes Hands With Every Passenger...,The 92-year-old former president boarded a Del...,0
6581,Chris Matthews : ‘ We Don ’ t Know What Sadnes...,That ’ s another reason I read : to recognize ...,1
6582,Ex-Twitter CEO : Meeting with Trump like water...,Former Twitter CEO Dick Costolo said President...,0


In [ ]:
# full_test_feat_flt = full_test_feat.reshape(full_test_feat.shape[0], full_test_feat.shape[1]*full_test_feat.shape[2])
# full_test_feat_flt = full_test_feat_flt[:, -1050:]
batch_size_test = 1024
pred_on_test = []
full_test_hs = []
classifier.eval()
with torch.no_grad():
  for i in range(0, test_raw.shape[0], batch_size_test):
      feat_on_tensor = test_raw[i:min(test_raw.shape[0], i+batch_size_test)]
      headlines = model.encode(feat_on_tensor['headline']).cuda()
      body_mask = feat_on_tensor['body'].apply(body_to_para)
      bert_ans, para_masks = bertify(body_mask)
      bert_ans.astype(np.float).astype(np.int).astype(np.bool).cuda()
      para_masks.astype(np.float).astype(np.int).astype(np.bool).cuda()
      y_pred = classifier(bert_ans, para_masks, headlines).cuda()
      for e in y_pred.detach().cpu().numpy():
          pred_on_test.append(e)
      torch.cuda.empty_cache()
# full_test_hs = np.array(full_test_hs)
print(len(pred_on_test))